In [1]:
PROJECT_ID = "rag-apps-440510"
LOCATION = "us-central1"

Retrieval-Augmented Generation (RAG) is a key advancement for Large Language Models (LLMs) for a couple of reasons:

Boosts Factual Accuracy: LLMs are trained on massive amounts of text data, but this data can be outdated or incomplete. RAG tackles this by allowing the LLM to access and incorporate relevant information from external sources during generation. This external fact-checking helps reduce made-up information, or "hallucinations," in the LLM's outputs, making them more trustworthy.

Enhances Relevance and Depth: RAG provides LLMs with a wider range of knowledge to draw on. When responding to a prompt or question, the LLM can not only use its internal knowledge but also supplement it with specific details retrieved from external data sources. This leads to more comprehensive and informative responses that are precisely tailored to the situation.

Overall, RAG elevates the credibility and usefulness of LLMs by ensuring their outputs are grounded in factual information and highly relevant to the context. This is crucial for applications like chatbots, educational tools, and even creative writing where factual grounding and rich detail are important.

#### Gemma
********

##### About Gemma

Gemma is a family of lightweight, state-of-the-art open models from Google, built from the same research and technology used to create the Gemini models. They are text-to-text, decoder-only large language models, available in English, with open weights, pre-trained variants, and instruction-tuned variants. Gemma models are well-suited for a variety of text generation tasks, including question answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as a laptop, desktop or your own cloud infrastructure, democratizing access to state of the art AI models and helping foster innovation for everyone.

##### Prompt formatting

Instruction-tuned (IT) models are trained with a specific formatter that annotates all instruction tuning examples with extra information, both at training and inference time. The formatter has two purposes:

Indicating roles in a conversation, such as the system, user, or assistant roles.
Delineating turns in a conversation, especially in a multi-turn conversation.
Below is the control tokens used by Gemma and their use cases. Note that the control tokens are reserved in and specific to our tokenizer.

Token to indicate a user turn: user
Token to indicate a model turn: model
Token to indicate the beginning of dialogue turn: <start_of_turn>
Token to indicate the end of dialogue turn: <end_of_turn>
Here's the official documentation regarding prompting instruction-tuned models.

In [ ]:
# install packages
%pip install -q langchain langchain-google-vertexai
%pip install -q langchainhub langchain-chroma langchain_community langchain-huggingface langchain-ollama
%pip install -q sentence-transformers==2.2.2
%pip install -q -U huggingface_hub
%pip install -U 

In [ ]:
import sys
import vertexai
# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth
    auth.authenticate_user()
    
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from IPython.display import display
from rich import print as rich_print
from rich.markdown import Markdown as rich_Markdown
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Image,
)
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import MultiModalEmbeddingModel
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
template = """
Question: {question}
Answer: Explain like I have never heard of Google Before
"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.1", max_tokens=100)
    
chain = prompt | model
chain.invoke("What is the Google DevFest?")

"The Google DevFest is a global event series organized by Google that brings together developers, technologists, and innovators to learn, share, and collaborate on various aspects of technology. These events focus on promoting innovation, community building, and skill development within the tech industry.\n\nHere are some key points about Google DevFests:\n\n1. **Global Reach**: DevFest events take place across the world in multiple countries. The exact locations change every year but aim to cover as much geographical territory as possible.\n2. **Developer Community Engagement**: One of the primary goals is to engage with local developer communities, providing them a platform to learn and share experiences related to Google technologies such as Android, Firebase, Flutter, Web Development (with technologies like Angular, React, Vue.js), Cloud Computing (Cloud Platform), Machine Learning (TensorFlow), and more.\n3. **Hands-on Learning**: DevFests are interactive events where attendees ca

In [38]:

import glob
import os
import time
from typing import Any, Dict, Iterable, List, Optional, Tuple, Union

from IPython.display import display
import PIL
import fitz
import numpy as np
import pandas as pd
import requests
from vertexai.generative_models import (
    GenerationConfig,
    HarmBlockThreshold,
    HarmCategory,
    Image,
)
from vertexai.vision_models import Image as vision_model_Image
from langchain_core.output_parsers import BaseTransformOutputParser

In [23]:
# function to set embeddings as global variable

def set_global_variable(variable_name: str, value: any) -> None:
    """
    Sets the value of a global variable.

    Args:
        variable_name: The name of the global variable (as a string).
        value: The value to assign to the global variable. This can be of any type.
    """
    global_vars = globals()  # Get a dictionary of global variables
    global_vars[variable_name] = value 



# Functions for getting text and image embeddings

def get_text_embedding_from_text_embedding_model(
    text: str,
    return_array: Optional[bool] = False,
) -> list:
    """
    Generates a numerical text embedding from a provided text input using a text embedding model.

    Args:
        text: The input text string to be embedded.
        return_array: If True, returns the embedding as a NumPy array.
                      If False, returns the embedding as a list. (Default: False)

    Returns:
        list or numpy.ndarray: A 768-dimensional vector representation of the input text.
                               The format (list or NumPy array) depends on the
                               value of the 'return_array' parameter.
    """
    embeddings = text_embedding_model.get_embeddings([text])
    text_embedding = [embedding.values for embedding in embeddings][0]

    if return_array:
        text_embedding = np.fromiter(text_embedding, dtype=float)

    # returns 768 dimensional array
    return text_embedding


def get_image_embedding_from_multimodal_embedding_model(
    image_uri: str,
    embedding_size: int = 512,
    text: Optional[str] = None,
    return_array: Optional[bool] = False,
) -> list:
    """Extracts an image embedding from a multimodal embedding model.
    The function can optionally utilize contextual text to refine the embedding.

    Args:
        image_uri (str): The URI (Uniform Resource Identifier) of the image to process.
        text (Optional[str]): Optional contextual text to guide the embedding generation. Defaults to "".
        embedding_size (int): The desired dimensionality of the output embedding. Defaults to 512.
        return_array (Optional[bool]): If True, returns the embedding as a NumPy array.
        Otherwise, returns a list. Defaults to False.

    Returns:
        list: A list containing the image embedding values. If `return_array` is True, returns a NumPy array instead.
    """
    # image = Image.load_from_file(image_uri)
    image = vision_model_Image.load_from_file(image_uri)
    embeddings = multimodal_embedding_model.get_embeddings(
        image=image, contextual_text=text, dimension=embedding_size
    )  # 128, 256, 512, 1408
    image_embedding = embeddings.image_embedding

    if return_array:
        image_embedding = np.fromiter(image_embedding, dtype=float)

    return image_embedding


def load_image_bytes(image_path):
    """Loads an image from a URL or local file path.

    Args:
        image_uri (str): URL or local file path to the image.

    Raises:
        ValueError: If `image_uri` is not provided.

    Returns:
        bytes: Image bytes.
    """
    # Check if the image_uri is provided
    if not image_path:
        raise ValueError("image_uri must be provided.")

    # Load the image from a weblink
    if image_path.startswith("http://") or image_path.startswith("https://"):
        response = requests.get(image_path, stream=True)
        if response.status_code == 200:
            return response.content

    # Load the image from a local path
    else:
        return open(image_path, "rb").read()


def get_pdf_doc_object(pdf_path: str) -> tuple[fitz.Document, int]:
    """
    Opens a PDF file using fitz.open() and returns the PDF document object and the number of pages.

    Args:
        pdf_path: The path to the PDF file.

    Returns:
        A tuple containing the `fitz.Document` object and the number of pages in the PDF.

    Raises:
        FileNotFoundError: If the provided PDF path is invalid.

    """

    # Open the PDF file
    doc: fitz.Document = fitz.open(pdf_path)

    # Get the number of pages in the PDF file
    num_pages: int = len(doc)

    return doc, num_pages


# Add colors to the print
class Color:
    """
    This class defines a set of color codes that can be used to print text in different colors.
    This will be used later to print citations and results to make outputs more readable.
    """

    PURPLE: str = "\033[95m"
    CYAN: str = "\033[96m"
    DARKCYAN: str = "\033[36m"
    BLUE: str = "\033[94m"
    GREEN: str = "\033[92m"
    YELLOW: str = "\033[93m"
    RED: str = "\033[91m"
    BOLD: str = "\033[1m"
    UNDERLINE: str = "\033[4m"
    END: str = "\033[0m"


def get_text_overlapping_chunk(
    text: str, character_limit: int = 1000, overlap: int = 100
) -> dict:
    """
    * Breaks a text document into chunks of a specified size, with an overlap between chunks to preserve context.
    * Takes a text document, character limit per chunk, and overlap between chunks as input.
    * Returns a dictionary where the keys are chunk numbers and the values are the corresponding text chunks.

    Args:
        text: The text document to be chunked.
        character_limit: Maximum characters per chunk (defaults to 1000).
        overlap: Number of overlapping characters between chunks (defaults to 100).

    Returns:
        A dictionary where keys are chunk numbers and values are the corresponding text chunks.

    Raises:
        ValueError: If `overlap` is greater than `character_limit`.

    """

    if overlap > character_limit:
        raise ValueError("Overlap cannot be larger than character limit.")

    # Initialize variables
    chunk_number = 1
    chunked_text_dict = {}

    # Iterate over text with the given limit and overlap
    for i in range(0, len(text), character_limit - overlap):
        end_index = min(i + character_limit, len(text))
        chunk = text[i:end_index]

        # Encode and decode for consistent encoding
        chunked_text_dict[chunk_number] = chunk.encode("ascii", "ignore").decode(
            "utf-8", "ignore"
        )

        # Increment chunk number
        chunk_number += 1

    return chunked_text_dict


def get_page_text_embedding(text_data: Union[dict, str]) -> dict:
    """
    * Generates embeddings for each text chunk using a specified embedding model.
    * Takes a dictionary of text chunks and an embedding size as input.
    * Returns a dictionary where the keys are chunk numbers and the values are the corresponding embeddings.

    Args:
        text_data: Either a dictionary of pre-chunked text or the entire page text.
        embedding_size: Size of the embedding vector (defaults to 128).

    Returns:
        A dictionary where keys are chunk numbers or "text_embedding" and values are the corresponding embeddings.

    """

    embeddings_dict = {}

    if not text_data:
        return embeddings_dict

    if isinstance(text_data, dict):
        # Process each chunk
        # print(text_data)
        for chunk_number, chunk_value in text_data.items():
            text_embd = get_text_embedding_from_text_embedding_model(text=chunk_value)
            embeddings_dict[chunk_number] = text_embd
    else:
        # Process the first 1000 characters of the page text
        text_embd = get_text_embedding_from_text_embedding_model(text=text_data)
        embeddings_dict["text_embedding"] = text_embd

    return embeddings_dict


def get_chunk_text_metadata(
    page: fitz.Page,
    character_limit: int = 1000,
    overlap: int = 100,
    embedding_size: int = 128,
) -> tuple[str, dict, dict, dict]:
    """
    * Extracts text from a given page object, chunks it, and generates embeddings for each chunk.
    * Takes a page object, character limit per chunk, overlap between chunks, and embedding size as input.
    * Returns the extracted text, the chunked text dictionary, and the chunk embeddings dictionary.

    Args:
        page: The fitz.Page object to process.
        character_limit: Maximum characters per chunk (defaults to 1000).
        overlap: Number of overlapping characters between chunks (defaults to 100).
        embedding_size: Size of the embedding vector (defaults to 128).

    Returns:
        A tuple containing:
            - Extracted page text as a string.
            - Dictionary of embeddings for the entire page text (key="text_embedding").
            - Dictionary of chunked text (key=chunk number, value=text chunk).
            - Dictionary of embeddings for each chunk (key=chunk number, value=embedding).

    Raises:
        ValueError: If `overlap` is greater than `character_limit`.

    """

    if overlap > character_limit:
        raise ValueError("Overlap cannot be larger than character limit.")

    # Extract text from the page
    text: str = page.get_text().encode("ascii", "ignore").decode("utf-8", "ignore")

    # Get whole-page text embeddings
    page_text_embeddings_dict: dict = get_page_text_embedding(text)

    # Chunk the text with the given limit and overlap
    chunked_text_dict: dict = get_text_overlapping_chunk(text, character_limit, overlap)
    # print(chunked_text_dict)

    # Get embeddings for the chunks
    chunk_embeddings_dict: dict = get_page_text_embedding(chunked_text_dict)
    # print(chunk_embeddings_dict)

    # Return all extracted data
    return text, page_text_embeddings_dict, chunked_text_dict, chunk_embeddings_dict


def get_image_for_gemini(
    doc: fitz.Document,
    image: tuple,
    image_no: int,
    image_save_dir: str,
    file_name: str,
    page_num: int,
) -> Tuple[Image, str]:
    """
    Extracts an image from a PDF document, converts it to JPEG format (handling color conversions), saves it, and loads it as a PIL Image Object.
    """

    xref = image[0]
    pix = fitz.Pixmap(doc, xref)

    # Check and convert color space if needed
    if pix.colorspace not in (fitz.csGRAY, fitz.csRGB, fitz.csCMYK):
        pix = fitz.Pixmap(fitz.csRGB, pix)  # Convert to RGB, which JPEG supports

    # Now save as JPEG (no need for pix.tobytes("jpeg"))
    image_name = f"{image_save_dir}/{file_name}_image_{page_num}_{image_no}_{xref}.jpeg"
    os.makedirs(image_save_dir, exist_ok=True)
    pix.save(image_name)

    image_for_gemini = Image.load_from_file(image_name)
    return image_for_gemini, image_name


def get_model_response(
    generative_multimodal_model,
    model_input: List[str],
    stream: bool = True,
    generation_config: Optional[GenerationConfig] = GenerationConfig(
        temperature=0.2, max_output_tokens=2048
    ),
    safety_settings: Optional[dict] = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
    print_exception: bool = False,
) -> str:
    """
    This function generates text in response to a list of model inputs.

    Args:
        model_input: A list of strings representing the inputs to the model.
        stream: Whether to generate the response in a streaming fashion (returning chunks of text at a time) or all at once. Defaults to False.

    Returns:
        The generated text as a string.
    """
    response = generative_multimodal_model.invoke(
        model_input,
        generation_config=generation_config,
        stream=stream,
        safety_settings=safety_settings,
    )
    response_list = []

    for chunk in response:
        print(chunk)
        try:
            response_list.append(chunk.text)
        except Exception as e:
            if print_exception:
              print(
                  "Exception occurred while calling gemini. Something is blocked. Lower the safety thresholds [safety_settings: BLOCK_NONE ] if not already done. -----",
                  e,
              )
            else:
              print("Exception occurred while calling g")
            response_list.append("**Something blocked.**")
            continue
    response = "".join(response_list)

    return response


def get_text_metadata_df(
    filename: str, text_metadata: Dict[Union[int, str], Dict]
) -> pd.DataFrame:
    """
    This function takes a filename and a text metadata dictionary as input,
    iterates over the text metadata dictionary and extracts the text, chunk text,
    and chunk embeddings for each page, creates a Pandas DataFrame with the
    extracted data, and returns it.

    Args:
        filename: The filename of the document.
        text_metadata: A dictionary containing the text metadata for each page.

    Returns:
        A Pandas DataFrame with the extracted text, chunk text, and chunk embeddings for each page.
    """

    final_data_text: List[Dict] = []

    for key, values in text_metadata.items():
        for chunk_number, chunk_text in values["chunked_text_dict"].items():
            data: Dict = {}
            data["file_name"] = filename
            data["page_num"] = int(key) + 1
            data["text"] = values["text"]
            data["text_embedding_page"] = values["page_text_embeddings"][
                "text_embedding"
            ]
            data["chunk_number"] = chunk_number
            data["chunk_text"] = chunk_text
            data["text_embedding_chunk"] = values["chunk_embeddings_dict"][chunk_number]

            final_data_text.append(data)

    return_df = pd.DataFrame(final_data_text)
    return_df = return_df.reset_index(drop=True)
    return return_df


def get_image_metadata_df(
    filename: str, image_metadata: Dict[Union[int, str], Dict]
) -> pd.DataFrame:
    """
    This function takes a filename and an image metadata dictionary as input,
    iterates over the image metadata dictionary and extracts the image path,
    image description, and image embeddings for each image, creates a Pandas
    DataFrame with the extracted data, and returns it.

    Args:
        filename: The filename of the document.
        image_metadata: A dictionary containing the image metadata for each page.

    Returns:
        A Pandas DataFrame with the extracted image path, image description, and image embeddings for each image.
    """

    final_data_image: List[Dict] = []
    for key, values in image_metadata.items():
        for _, image_values in values.items():
            data: Dict = {}
            data["file_name"] = filename
            data["page_num"] = int(key) + 1
            data["img_num"] = int(image_values["img_num"])
            data["img_path"] = image_values["img_path"]
            data["img_desc"] = image_values["img_desc"]
            # data["mm_embedding_from_text_desc_and_img"] = image_values[
            #     "mm_embedding_from_text_desc_and_img"
            # ]
            data["mm_embedding_from_img_only"] = image_values[
                "mm_embedding_from_img_only"
            ]
            data["text_embedding_from_image_description"] = image_values[
                "text_embedding_from_image_description"
            ]
            final_data_image.append(data)

    return_df = pd.DataFrame(final_data_image).dropna()
    return_df = return_df.reset_index(drop=True)
    return return_df


def get_document_metadata(
    generative_multimodal_model,
    pdf_folder_path: str,
    image_save_dir: str,
    image_description_prompt: str,
    embedding_size: int = 128,
    generation_config: Optional[GenerationConfig] = GenerationConfig(
        temperature=0.2, max_output_tokens=2048
    ),
    safety_settings: Optional[dict] = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
    add_sleep_after_page: bool = False,
    sleep_time_after_page: int = 2,
    add_sleep_after_document: bool = False,
    sleep_time_after_document: int = 2,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    This function takes a PDF path, an image save directory, an image description prompt, an embedding size, and a text embedding text limit as input.

    Args:
        pdf_path: The path to the PDF document.
        image_save_dir: The directory where extracted images should be saved.
        image_description_prompt: A prompt to guide Gemini for generating image descriptions.
        embedding_size: The dimensionality of the embedding vectors.
        text_emb_text_limit: The maximum number of tokens for text embedding.

    Returns:
        A tuple containing two DataFrames:
            * One DataFrame containing the extracted text metadata for each page of the PDF, including the page text, chunked text dictionaries, and chunk embedding dictionaries.
            * Another DataFrame containing the extracted image metadata for each image in the PDF, including the image path, image description, image embeddings (with and without context), and image description text embedding.
    """

    text_metadata_df_final, image_metadata_df_final = pd.DataFrame(), pd.DataFrame()

    for pdf_path in glob.glob(pdf_folder_path + "/*.pdf"):
        print(
            "\n\n",
            "Processing the file: ---------------------------------",
            pdf_path,
            "\n\n",
        )

        doc, num_pages = get_pdf_doc_object(pdf_path)

        file_name = pdf_path.split("/")[-1]

        text_metadata: Dict[Union[int, str], Dict] = {}

        for page_num in range(num_pages):
            print(f"Processing page: {page_num + 1}")

            page = doc[page_num]

            text = page.get_text()
            (
                text,
                page_text_embeddings_dict,
                chunked_text_dict,
                chunk_embeddings_dict,
            ) = get_chunk_text_metadata(page, embedding_size=embedding_size)

            text_metadata[page_num] = {
                "text": text,
                "page_text_embeddings": page_text_embeddings_dict,
                "chunked_text_dict": chunked_text_dict,
                "chunk_embeddings_dict": chunk_embeddings_dict,
            }

           
        text_metadata_df = get_text_metadata_df(file_name, text_metadata)

        text_metadata_df_final = pd.concat(
            [text_metadata_df_final, text_metadata_df], axis=0
        )
        
        text_metadata_df_final = text_metadata_df_final.reset_index(drop=True)

    return text_metadata_df_final


# Helper Functions


def get_user_query_text_embeddings(user_query: str) -> np.ndarray:
    """
    Extracts text embeddings for the user query using a text embedding model.

    Args:
        user_query: The user query text.
        embedding_size: The desired embedding size.

    Returns:
        A NumPy array representing the user query text embedding.
    """

    return get_text_embedding_from_text_embedding_model(user_query)


def get_user_query_image_embeddings(
    image_query_path: str, embedding_size: int
) -> np.ndarray:
    """
    Extracts image embeddings for the user query image using a multimodal embedding model.

    Args:
        image_query_path: The path to the user query image.
        embedding_size: The desired embedding size.

    Returns:
        A NumPy array representing the user query image embedding.
    """

    return get_image_embedding_from_multimodal_embedding_model(
        image_uri=image_query_path, embedding_size=embedding_size
    )


def get_cosine_score(
    dataframe: pd.DataFrame, column_name: str, input_text_embd: np.ndarray
) -> float:
    """
    Calculates the cosine similarity between the user query embedding and the dataframe embedding for a specific column.

    Args:
        dataframe: The pandas DataFrame containing the data to compare against.
        column_name: The name of the column containing the embeddings to compare with.
        input_text_embd: The NumPy array representing the user query embedding.

    Returns:
        The cosine similarity score (rounded to two decimal places) between the user query embedding and the dataframe embedding.
    """

    text_cosine_score = round(np.dot(dataframe[column_name], input_text_embd), 2)
    return text_cosine_score


def print_text_to_image_citation(
    final_images: Dict[int, Dict[str, Any]], print_top: bool = True
) -> None:
    """
    Prints a formatted citation for each matched image in a dictionary.

    Args:
        final_images: A dictionary containing information about matched images,
                    with keys as image number and values as dictionaries containing
                    image path, page number, page text, cosine similarity score, and image description.
        print_top: A boolean flag indicating whether to only print the first citation (True) or all citations (False).

    Returns:
        None (prints formatted citations to the console).
    """

    color = Color()

    # Iterate through the matched image citations
    for imageno, image_dict in final_images.items():
        # Print the citation header
        print(
            color.RED + f"Citation {imageno + 1}:",
            "Matched image path, page number and page text: \n" + color.END,
        )

        # Print the cosine similarity score
        print(color.BLUE + "score: " + color.END, image_dict["cosine_score"])

        # Print the file_name
        print(color.BLUE + "file_name: " + color.END, image_dict["file_name"])

        # Print the image path
        print(color.BLUE + "path: " + color.END, image_dict["img_path"])

        # Print the page number
        print(color.BLUE + "page number: " + color.END, image_dict["page_num"])

        # Print the page text
        print(
            color.BLUE + "page text: " + color.END, "\n".join(image_dict["page_text"])
        )

        # Print the image description
        print(
            color.BLUE + "image description: " + color.END,
            image_dict["image_description"],
        )

        # Only print the first citation if print_top is True
        if print_top and imageno == 0:
            break


def print_text_to_text_citation(
    final_text: Dict[int, Dict[str, Any]],
    print_top: bool = True,
    chunk_text: bool = True,
) -> None:
    """
    Prints a formatted citation for each matched text in a dictionary.

    Args:
        final_text: A dictionary containing information about matched text passages,
                    with keys as text number and values as dictionaries containing
                    page number, cosine similarity score, chunk number (optional),
                    chunk text (optional), and page text (optional).
        print_top: A boolean flag indicating whether to only print the first citation (True) or all citations (False).
        chunk_text: A boolean flag indicating whether to print individual text chunks (True) or the entire page text (False).

    Returns:
        None (prints formatted citations to the console).
    """

    color = Color()

    # Iterate through the matched text citations
    for textno, text_dict in final_text.items():
        # Print the citation header
        print(color.RED + f"Citation {textno + 1}:", "Matched text: \n" + color.END)

        # Print the cosine similarity score
        print(color.BLUE + "score: " + color.END, text_dict["cosine_score"])

        # Print the file_name
        print(color.BLUE + "file_name: " + color.END, text_dict["file_name"])

        # Print the page number
        print(color.BLUE + "page_number: " + color.END, text_dict["page_num"])

        # Print the matched text based on the chunk_text argument
        if chunk_text:
            # Print chunk number and chunk text
            print(color.BLUE + "chunk_number: " + color.END, text_dict["chunk_number"])
            print(color.BLUE + "chunk_text: " + color.END, text_dict["chunk_text"])
        else:
            # Print page text
            print(color.BLUE + "page text: " + color.END, text_dict["page_text"])

        # Only print the first citation if print_top is True
        if print_top and textno == 0:
            break


def get_similar_image_from_query(
    text_metadata_df: pd.DataFrame,
    image_metadata_df: pd.DataFrame,
    query: str = "",
    image_query_path: str = "",
    column_name: str = "",
    image_emb: bool = True,
    top_n: int = 3,
    embedding_size: int = 128,
) -> Dict[int, Dict[str, Any]]:
    """
    Finds the top N most similar images from a metadata DataFrame based on a text query or an image query.

    Args:
        text_metadata_df: A Pandas DataFrame containing text metadata associated with the images.
        image_metadata_df: A Pandas DataFrame containing image metadata (paths, descriptions, etc.).
        query: The text query used for finding similar images (if image_emb is False).
        image_query_path: The path to the image used for finding similar images (if image_emb is True).
        column_name: The column name in the image_metadata_df containing the image embeddings or captions.
        image_emb: Whether to use image embeddings (True) or text captions (False) for comparisons.
        top_n: The number of most similar images to return.
        embedding_size: The dimensionality of the image embeddings (only used if image_emb is True).

    Returns:
        A dictionary containing information about the top N most similar images, including cosine scores, image objects, paths, page numbers, text excerpts, and descriptions.
    """
    # Check if image embedding is used
    if image_emb:
        # Calculate cosine similarity between query image and metadata images
        user_query_image_embedding = get_user_query_image_embeddings(
            image_query_path, embedding_size
        )
        cosine_scores = image_metadata_df.apply(
            lambda x: get_cosine_score(x, column_name, user_query_image_embedding),
            axis=1,
        )
    else:
        # Calculate cosine similarity between query text and metadata image captions
        user_query_text_embedding = get_user_query_text_embeddings(query)
        cosine_scores = image_metadata_df.apply(
            lambda x: get_cosine_score(x, column_name, user_query_text_embedding),
            axis=1,
        )

    # Remove same image comparison score when user image is matched exactly with metadata image
    cosine_scores = cosine_scores[cosine_scores < 1.0]

    # Get top N cosine scores and their indices
    top_n_cosine_scores = cosine_scores.nlargest(top_n).index.tolist()
    top_n_cosine_values = cosine_scores.nlargest(top_n).values.tolist()

    # Create a dictionary to store matched images and their information
    final_images: Dict[int, Dict[str, Any]] = {}

    for matched_imageno, indexvalue in enumerate(top_n_cosine_scores):
        # Create a sub-dictionary for each matched image
        final_images[matched_imageno] = {}

        # Store cosine score
        final_images[matched_imageno]["cosine_score"] = top_n_cosine_values[
            matched_imageno
        ]

        # Load image from file
        final_images[matched_imageno]["image_object"] = Image.load_from_file(
            image_metadata_df.iloc[indexvalue]["img_path"]
        )

        # Add file name
        final_images[matched_imageno]["file_name"] = image_metadata_df.iloc[indexvalue][
            "file_name"
        ]

        # Store image path
        final_images[matched_imageno]["img_path"] = image_metadata_df.iloc[indexvalue][
            "img_path"
        ]

        # Store page number
        final_images[matched_imageno]["page_num"] = image_metadata_df.iloc[indexvalue][
            "page_num"
        ]

        final_images[matched_imageno]["page_text"] = np.unique(
            text_metadata_df[
                (
                    text_metadata_df["page_num"].isin(
                        [final_images[matched_imageno]["page_num"]]
                    )
                )
                & (
                    text_metadata_df["file_name"].isin(
                        [final_images[matched_imageno]["file_name"]]
                    )
                )
            ]["text"].values
        )

        # Store image description
        final_images[matched_imageno]["image_description"] = image_metadata_df.iloc[
            indexvalue
        ]["img_desc"]

    return final_images


def get_similar_text_from_query(
    query: str,
    text_metadata_df: pd.DataFrame,
    column_name: str = "",
    top_n: int = 3,
    chunk_text: bool = True,
    print_citation: bool = False,
) -> Dict[int, Dict[str, Any]]:
    """
    Finds the top N most similar text passages from a metadata DataFrame based on a text query.

    Args:
        query: The text query used for finding similar passages.
        text_metadata_df: A Pandas DataFrame containing the text metadata to search.
        column_name: The column name in the text_metadata_df containing the text embeddings or text itself.
        top_n: The number of most similar text passages to return.
        embedding_size: The dimensionality of the text embeddings (only used if text embeddings are stored in the column specified by `column_name`).
        chunk_text: Whether to return individual text chunks (True) or the entire page text (False).
        print_citation: Whether to immediately print formatted citations for the matched text passages (True) or just return the dictionary (False).

    Returns:
        A dictionary containing information about the top N most similar text passages, including cosine scores, page numbers, chunk numbers (optional), and chunk text or page text (depending on `chunk_text`).

    Raises:
        KeyError: If the specified `column_name` is not present in the `text_metadata_df`.
    """

    if column_name not in text_metadata_df.columns:
        raise KeyError(f"Column '{column_name}' not found in the 'text_metadata_df'")

    query_vector = get_user_query_text_embeddings(query)

    # Calculate cosine similarity between query text and metadata text
    cosine_scores = text_metadata_df.apply(
        lambda row: get_cosine_score(
            row,
            column_name,
            query_vector,
        ),
        axis=1,
    )

    # Get top N cosine scores and their indices
    top_n_indices = cosine_scores.nlargest(top_n).index.tolist()
    top_n_scores = cosine_scores.nlargest(top_n).values.tolist()

    # Create a dictionary to store matched text and their information
    final_text: Dict[int, Dict[str, Any]] = {}

    for matched_textno, index in enumerate(top_n_indices):
        # Create a sub-dictionary for each matched text
        final_text[matched_textno] = {}

        # Store page number
        final_text[matched_textno]["file_name"] = text_metadata_df.iloc[index][
            "file_name"
        ]

        # Store page number
        final_text[matched_textno]["page_num"] = text_metadata_df.iloc[index][
            "page_num"
        ]

        # Store cosine score
        final_text[matched_textno]["cosine_score"] = top_n_scores[matched_textno]

        if chunk_text:
            # Store chunk number
            final_text[matched_textno]["chunk_number"] = text_metadata_df.iloc[index][
                "chunk_number"
            ]

            # Store chunk text
            final_text[matched_textno]["chunk_text"] = text_metadata_df["chunk_text"][
                index
            ]
        else:
            # Store page text
            final_text[matched_textno]["text"] = text_metadata_df["text"][index]

    # Optionally print citations immediately
    if print_citation:
        print_text_to_text_citation(final_text, chunk_text=chunk_text)

    return final_text


def display_images(
    images: Iterable[Union[str, PIL.Image.Image]], resize_ratio: float = 0.5
) -> None:
    """
    Displays a series of images provided as paths or PIL Image objects.

    Args:
        images: An iterable of image paths or PIL Image objects.
        resize_ratio: The factor by which to resize each image (default 0.5).

    Returns:
        None (displays images using IPython or Jupyter notebook).
    """

    # Convert paths to PIL images if necessary
    pil_images = []
    for image in images:
        if isinstance(image, str):
            pil_images.append(PIL.Image.open(image))
        else:
            pil_images.append(image)

    # Resize and display each image
    for img in pil_images:
        original_width, original_height = img.size
        new_width = int(original_width * resize_ratio)
        new_height = int(original_height * resize_ratio)
        resized_img = img.resize((new_width, new_height))
        display(resized_img)
        print("\n")
        
def get_answer_from_qa_system(
    query: str,
    text_metadata_df,
    image_metadata_df,
    top_n_text: int = 10,
    top_n_image: int = 5,
    instruction: Optional[str] = None,
    model=None,
    generation_config: Optional[GenerationConfig] = GenerationConfig(
        temperature=1, max_output_tokens=8192
    ),
    safety_settings: Optional[dict] = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
) -> Union[str, None]:
    """Fetches answers from a combined text and image-based QA system.

    Args:
        query (str): The user's question.
        text_metadata_df: DataFrame containing text embeddings, file names, and page numbers.
        image_metadata_df: DataFrame containing image embeddings, paths, and descriptions.
        top_n_text (int, optional): Number of top text chunks to consider. Defaults to 10.
        top_n_image (int, optional): Number of top images to consider. Defaults to 5.
        instruction (str, optional): Customized instruction for the model. Defaults to a generic one.
        model: Model to use for QA.
        safety_settings: Safety settings for the model.
        generation_config: Generation configuration for the model.

    Returns:
        Union[str, None]: The generated answer or None if an error occurs.
    """
    # Build Gemini content
    if instruction is None:  # Use default instruction if not provided
        instruction = """Task: Answer the following questions in detail, providing clear reasoning and evidence from the images and text in bullet points.
                      Instructions:

                      1. **Analyze:** Carefully examine the provided images and text context.
                      2. **Synthesize:** Integrate information from both the visual and textual elements.
                      3. **Reason:**  Deduce logical connections and inferences to address the question.
                      4. **Respond:** Provide a concise, accurate answer in the following format:

                        * **Question:** [Question]
                        * **Answer:** [Direct response to the question]
                        * **Explanation:** [Bullet-point reasoning steps if applicable]
                        * **Source** [name of the file, page, image from where the information is citied]

                      5. **Ambiguity:** If the context is insufficient to answer, respond "Not enough context to answer."

                      """

    # Retrieve relevant chunks of text based on the query
    matching_results_chunks_data = get_similar_text_from_query(
        query,
        text_metadata_df,
        column_name="text_embedding_chunk",
        top_n=top_n_text,
        chunk_text=True,
    )
    # Get all relevant images based on user query
    matching_results_image_fromdescription_data = get_similar_image_from_query(
        text_metadata_df,
        image_metadata_df,
        query=query,
        column_name="text_embedding_from_image_description",
        image_emb=False,
        top_n=top_n_image,
        embedding_size=1408,
    )

    # combine all the selected relevant text chunks
    context_text = ["Text Context: "]
    for key, value in matching_results_chunks_data.items():
        context_text.extend(
            [
                "Text Source: ",
                f"""file_name: "{value["file_name"]}" Page: "{value["page_num"]}""",
                "Text",
                value["chunk_text"],
            ]
        )

    # combine all the selected relevant images
    gemini_content = [
        instruction,
        "Questions: ",
        query,
        "Image Context: ",
    ]
    for key, value in matching_results_image_fromdescription_data.items():
        gemini_content.extend(
            [
                "Image Path: ",
                value["img_path"],
                "Image Description: ",
                value["image_description"],
                "Image:",
                value["image_object"],
            ]
        )
    gemini_content.extend(context_text)

    # Get Gemini response with streaming (if supported)
    response = get_model_response(
        model,
        model_input=gemini_content,
        stream=True,
        safety_settings=safety_settings,
        generation_config=generation_config,
    )

    return (
        response,
        matching_results_chunks_data,
        matching_results_image_fromdescription_data,
    )


In [17]:
# Load text embedding model from pre-trained source
text_embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")

# Load multimodal embedding model from pre-trained source
multimodal_embedding_model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding")  # works with image, image with caption(~32 words), video, video with caption(~32 words)
     

In [24]:
set_global_variable("text_embedding_model", text_embedding_model)
set_global_variable("multimodal_embedding_model", multimodal_embedding_model)

In [25]:
pdf_folder_path = "data/"  # if running in Vertex AI Workbench.

# Specify the image description prompt. Change it
# image_description_prompt = """Explain what is going on in the image.
# If it's a table, extract all elements of the table.
# If it's a graph, explain the findings in the graph.
# Do not include any numbers that are not mentioned in the image.
# """

image_description_prompt = """You are a technical image analysis expert. You will be provided with various types of images extracted from documents like research papers, technical blogs, and more.
Your task is to generate concise, accurate descriptions of the images without adding any information you are not confident about.
Focus on capturing the key details, trends, or relationships depicted in the image.

Important Guidelines:
* Prioritize accuracy:  If you are uncertain about any detail, state "Unknown" or "Not visible" instead of guessing.
* Avoid hallucinations: Do not add information that is not directly supported by the image.
* Be specific: Use precise language to describe shapes, colors, textures, and any interactions depicted.
* Consider context: If the image is a screenshot or contains text, incorporate that information into your description.
"""

In [26]:
# Extract text and image metadata from the PDF document
text_metadata_df = get_document_metadata( #image_metadata_df
    gemma2_model,  # we are passing Gemini 1.5 Pro
    pdf_folder_path,
    image_save_dir="images",
    image_description_prompt=image_description_prompt,
    embedding_size=1408,
    # add_sleep_after_page = True, # Uncomment this if you are running into API quota issues
    # sleep_time_after_page = 5,
    add_sleep_after_document=True,  # Uncomment this if you are running into API quota issues
    sleep_time_after_document=5,  # Increase the value in seconds, if you are still getting quota issues. It will slow down the processing.
    # generation_config = # see next cell
    # safety_settings =  # see next cell
)
print("\n\n --- Completed processing. ---")



 Processing the file: --------------------------------- data/gemma_technical_paper.pdf 


Processing page: 1
Processing page: 2
Processing page: 3
Processing page: 4
Processing page: 5
Processing page: 6
Processing page: 7
Processing page: 8
Processing page: 9
Processing page: 10
Processing page: 11
Processing page: 12
Processing page: 13
Processing page: 14
Processing page: 15
Processing page: 16
Processing page: 17


 --- Completed processing. ---


In [1]:
text_metadata_df

NameError: name 'text_metadata_df' is not defined

In [27]:
text_metadata_df

,file_name,page_num,text,text_embedding_page,chunk_number,chunk_text,text_embedding_chunk
0,gemma_technical_paper.pdf,1,2024-02-21\nGemma: Open Models Based on Gemini...,"[0.029665455222129822, 0.043536581099033356, -...",1,2024-02-21\nGemma: Open Models Based on Gemini...,"[0.035918254405260086, 0.039294395595788956, -..."
1,gemma_technical_paper.pdf,1,2024-02-21\nGemma: Open Models Based on Gemini...,"[0.029665455222129822, 0.043536581099033356, -...",2,"Gemma, a family of open models\nbased on Googl...","[0.03788634389638901, 0.05161239951848984, -0...."
2,gemma_technical_paper.pdf,1,2024-02-21\nGemma: Open Models Based on Gemini...,"[0.029665455222129822, 0.043536581099033356, -...",3,"d for dialogue, instruction-following, help-\n...","[0.045551132410764694, 0.04680870845913887, -0..."
3,gemma_technical_paper.pdf,1,2024-02-21\nGemma: Open Models Based on Gemini...,"[0.029665455222129822, 0.043536581099033356, -...",4,"ce (Cobbe et al.,\n2021; Hendrycks et al., 202...","[0.017123950645327568, 0.05736316367983818, -0..."
4,gemma_technical_paper.pdf,1,2024-02-21\nGemma: Open Models Based on Gemini...,"[0.029665455222129822, 0.043536581099033356, -...",5,rigorous evaluation and\nanalysis of current ...,"[0.03978141397237778, 0.029347488656640053, -0..."
...,...,...,...,...,...,...,...
63,gemma_technical_paper.pdf,15,Gemma: Open Models Based on Gemini Research an...,"[-0.02015767991542816, 0.046027496457099915, -...",5,"g, M. Cheng, M. Glaese,\nB. Balle, A. Kasirzad...","[-0.0030858826357871294, 0.009666250087320805,..."
64,gemma_technical_paper.pdf,16,Gemma: Open Models Based on Gemini Research an...,"[0.022678498178720474, 0.04704996943473816, -0...",1,Gemma: Open Models Based on Gemini Research an...,"[0.028383769094944, 0.05077578127384186, -0.04..."
65,gemma_technical_paper.pdf,16,Gemma: Open Models Based on Gemini Research an...,"[0.022678498178720474, 0.04704996943473816, -0...",2,"J. E. Gonzalez, and I. Stoica. Judg-\ning llm-...","[0.014260362833738327, 0.01989196427166462, -0..."
66,gemma_technical_paper.pdf,17,Gemma: Open Models Based on Gemini Research an...,"[0.047216158360242844, 0.059248290956020355, -...",1,Gemma: Open Models Based on Gemini Research an...,"[0.05735192447900772, 0.0586150586605072, -0.0..."


In [29]:
# Matching user text query with "chunk_embedding" to find relevant chunks.
query = "What is the safety percentage?"

matching_results_text = get_similar_text_from_query(
    query,
    text_metadata_df,
    column_name="text_embedding_chunk",
    top_n=3,
    chunk_text=True,
)

# Print the matched text citations
print_text_to_text_citation(
    matching_results_text, print_top=True, chunk_text=True
)  # print_top=False to see all text matches

Citation 1: Matched text: 

score:  0.46
file_name:  gemma_technical_paper.pdf
page_number:  17
chunk_number:  1
chunk_text:  Gemma: Open Models Based on Gemini Research and Technology
Gemma 1.0 IT results
The core of the paper presents the results of the Gemma 1.1 IT models. We kept the results of the
previous Gemma 1.0 IT models for comparison in this appendix. Side-by-side evaluations of Gemma
1.0 IT against Mistral 7b v0.2 can be found in table 9. Safety academic benchmark results of version
1.0 can be found in table 10.
Model
Safety
Instruction Following
Gemma 7B IT
58%
51.7%
95% Conf. Interval
[55.9%, 60.1%]
[49.6%, 53.8%]
Win / Tie / Loss
42.9% / 30.2% / 26.9%
42.5% / 18.4% / 39.1%
Gemma 2B IT
56.5%
41.6%
95% Conf. Interval
[54.4%, 58.6%]
[39.5%, 43.7%]
Win / Tie / Loss
44.8% / 22.9% / 32.3%
32.7% / 17.8% / 49.5%
Table 9 | Win rate of Gemma 1.0 IT models versus Mistral 7B v0.2 Instruct with 95% confidence
intervals. We report breakdowns of wins, ties, and losses. Ties are broken

In [39]:
# Helpers
class GemmaOutputParser(BaseTransformOutputParser[str]):
    """OutputParser that parses LLM response and extract
    the generated part."""

    @classmethod
    def is_lc_serializable(cls) -> bool:
        """Return whether this class is serializable."""
        return True

    @property
    def _type(self) -> str:
        """Return the output parser type for serialization."""
        return "gemma_2_parser"

    def parse(self, text: str) -> str:
        """Return the input text with no changes."""
        model_start_token = "<start_of_turn>model\n"
        idx = text.rfind(model_start_token)
        return text[idx + len(model_start_token) :] if idx > -1 else ""


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [75]:
# Assuming `matching_results_text` and `query` are defined earlier
context = "\n".join(
    [value["chunk_text"] for key, value in matching_results_text.items()]
)

template = f"""
Question: {{question}}
Context: {{context}}
Answer: Answer the question with the given context. If the specific answer is not in the context, please answer "I don't know".
"""

# Define a prompt template with placeholders `{question}` and `{context}`
prompt = ChatPromptTemplate.from_template(template)

# Set up the query and context in a dictionary to pass into the chain
input_data = {
    "question": "What is the Document About",
    "context": context
}

# Invoke the chain with the input data
rag_chain = prompt | model #| GemmaOutputParser()

output = rag_chain.invoke(input_data)

print(output)

This document describes the Gemma 1.1 open-source language models based on research and technology from Gemini.  It focuses on the results of these models, comparing them to Mistral 7B v0.2 in various benchmarks like safety and instruction following. The document also touches upon responsible deployment practices followed by the Gemma team.  





In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [51]:
loader = PyPDFLoader("data/gemma_technical_paper.pdf")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [ ]:
# Create a vector store with all the docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(pages)
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [40]:
# Let's load a predefined prompt for this task
prompt = hub.pull("rlm/rag-prompt")
print(f"Prompt:\n\n{prompt.messages[0].prompt.template}")

/home/saol/miniconda3/envs/llm/lib/python3.10/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Prompt:

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [43]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# Create an actual chain

rag_chain = (
    # First you need retrieve documents that are relevant to the
    # given query
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    # The output is passed the prompt and fills fields like `{question}`
    # and `{context}`
    | prompt
    # The whole prompt will all the information is passed the LLM
    | gemma2_model
    # # The answer of the LLM is parsed by the class defined above
    # | GemmaOutputParser()
)

In [ ]:
rag_chain.invoke("What are transformers?")

In [67]:
rag_chain = (
    # First you need retrieve documents that are relevant to the
    # given query
    {"context": {context}, "question": {query}}
    # The output is passed the prompt and fills fields like `{question}`
    # and `{context}`
    | prompt
    # The whole prompt will all the information is passed the LLM
    | gemma2_model
)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'set'>

In [ ]:
rag_chain.invoke("What is the Document About")

In [99]:
safety_settings = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
}

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

gemma2_model = OllamaLLM(model="gemma2")
    
chain = prompt | gemma2_model

In [ ]:
%%time
# Generate response with Gemini 1.5 Pro
print("\n **** Result: ***** \n")

rich_Markdown(
    get_model_response(
        chain,
        model_input=chain,
        # stream=True,
        # safety_settings=safety_settings,
        # generation_config=GenerationConfig(temperature=1, max_output_tokens=8192),
    )
)

In [24]:
multimodal_model_15_flash = GenerativeModel(
    "gemini-1.5-flash"
)  # works with text, code, images, video(with or without audio) and audio(mp3)

In [ ]:
credentials, _ = google.auth.default(quota_project_id=PROJECT_ID)
embeddings = VertexAIEmbeddings(
    project=PROJECT_ID, model_name="text-embedding-004", credentials=credentials
)

In [ ]:
# model = genai.GenerativeModel("gemini-1.5-flash")
response = multimodal_model_15_flash.generate_content("Write a story about a magic backpack.")
print(response.text)

In [50]:
import bs4
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import BaseTransformOutputParser
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

from tqdm.autonotebook import tqdm
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

In [6]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://developers.google.com/machine-learning/resources/intro-llms",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(name=("h3", "p"))
    ),
)
docs = loader.load()

In [8]:
# Create a vector store with all the docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [ ]:
# Let's load a predefined prompt for this task
prompt = hub.pull("rlm/rag-prompt")
print(f"Prompt:\n\n{prompt.messages[0].prompt.template}")

In [ ]:
# Create an actual chain

rag_chain = (
    # First you need retrieve documents that are relevant to the
    # given query
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    # The output is passed the prompt and fills fields like `{question}`
    # and `{context}`
    | prompt
    # The whole prompt will all the information is passed the LLM
    | gemma2_model #llm
    # The answer of the LLM is parsed by the class defined above
    | GemmaOutputParser()
)

In [ ]:
rag_chain.invoke("What are transformers?")

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/gtr-t5-xxl')

In [ ]:
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/gtr-t5-xxl')
embeddings = model.encode(sentences)
print(embeddings)


[[-0.00880101 -0.0460175   0.00308607 ... -0.01346269  0.0163978
  -0.03259923]
 [ 0.00987251 -0.06034163 -0.01103296 ...  0.00029951  0.0309834
  -0.02386388]]


In [ ]:
<script>
                document.getElementById("mainForm").addEventListener("submit", async function(event) {
                    event.preventDefault();
                    const formData = new FormData(event.target);
                    const fileInput = document.getElementById("fileInput");
                    const questionInput = document.getElementById("question");

                    if (fileInput.files.length > 0) {
                        // Post to upload_documents if files are provided
                        await fetch('/upload_documents', {
                            method: 'POST',
                            body: formData
                        });

                        // Start polling for processing status
                        pollProcessingStatus();
                        
                        const response =  await fetch('/query_documents', {
                            method: 'POST',
                            body: formData
                        });
                        
                        const data = await response.json();
                    } else if(questionInput.value) {
                        // Post directly to /chain if no files are uploaded
                        const response = await fetch('/chain', {
                            method: 'POST',
                            body: formData
                        });
                        const data = await response.json();
                        document.getElementById("statusText").textContent = data.message;
                    } else{
                        JSON.error("Please enter a question or upload files.");
                    }
                });

                async function pollProcessingStatus() {
                    const statusText = document.getElementById("statusText");
                    const loadingBarProgress = document.getElementById("loadingBarProgress");

                    // Disable input fields during processing
                    document.getElementById("question").disabled = true;
                    document.getElementById("model").disabled = true;
                    document.getElementById("answer_format").disabled = true;
                    document.getElementById("fileInput").disabled = true;
                    
                    const interval = setInterval(async () => {
                        const response = await fetch('/processing_status');
                        const status = await response.json();

                        // Update progress bar and status text
                        loadingBarProgress.style.width = status.progress + '%';
                        loadingBarProgress.textContent = status.progress + '%';
                        statusText.textContent = `Processing ${status.current_file} - ${status.current_page}`;

                        if (!status.in_progress) {
                            clearInterval(interval);
                            statusText.textContent = "Processing complete.";
                            document.getElementById("question").disabled = false;
                        }
                    }, 1000);
                }
            </script>